# Data-driven PBRPK for Theranostics
Author: Binesh Sadanandan
Date created: 2024/02
Description: In this study, we are trying to model a data driven PBPK model based on https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10803696/


# Install required python packages

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC,SVR
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
import sys
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import Ridge


pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


## Read data from Drive 

In [ ]:
def process_csv_files(folder_path):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    dataframes = []
    print(f'Found {len(csv_files)} csv files in {folder_path}')

    for file in csv_files:
        #print(f'Processing {file}......')
        try:
         df = pd.read_csv(os.path.join(folder_path, file))
         df['FileName'] = file
         df.columns = [col.replace('UnknownCompartment_', '') for col in df.columns]
         dataframes.append(df)
        except Exception as e:
            print(f'Error processing {file} : {e}')
            continue
    df_all = pd.concat(dataframes, ignore_index=True)

    for col in df_all.columns:
        if col != 'FileName':
            df_all[col] = df_all[col].astype(float)
    
    return df_all


In [ ]:
folder_path = '/Users/tbin/Library/CloudStorage/GoogleDrive-contact@bineshkumar.me/My Drive/Colab Notebooks/UBC_PSMA_Model_Simulator_Data/SimDataCSVs'
# Uncomment the below line to read all the files 
df_all=process_csv_files(folder_path)


In [ ]:
[col for col in df_all.columns if 'bser'  in col or 'parameter' in col or ('Vein' in col and 'Alb' not in col) or 'Time' in col]

In [ ]:
cols=['FileName','Time',
 'Vein_species_Hot',
 'Vein_species_Cold',
 'parameter_lambdaPhys',
 'parameter_numberPerNanomole',
 'observable_repeatInterval',
 'observable_repeatdose',
 'parameter_Tumor1VolumeCoeff',
 'parameter_bodyWeight',
 'parameter_bodySurfaceArea',
 'observable_Tumor1_TotalHot',
 'observable_Tumor2_TotalHot',
 'observable_TumorTotalHot',
 'observable_KidneyTotalHot',
 'observable_SGTotalHot',
 'observable_LiverTotalHot',
 'observable_ProstateTotalHot',
 'observable_SpleenTotalHot',
 'observable_RedMarrowTotalHot',
 'observable_BoneTotalHot',
 'observable_TIA_TumorTotal',
 'observable_TIA_Kidney',
 'observable_TIA_SG',
 'observable_TIA_Liver',
 'observable_TIA_Spleen',
 'observable_TIA_RedMarrow',
 'observable_HotStuffBlood',
 'observable_BloodTIA']

In [ ]:
#df_all.to_csv(folder_path.replace('SimDataCSVs', 'df_all_sim.csv'))

## Exploratory Data Analysis

In [ ]:
df = pd.read_csv(folder_path.replace('SimDataCSVs', 'df_all_sim.csv'))
df_all=df_all[df_all['Time']<11520]

In [ ]:
def filter_df_by_unique_values(df, column_name, num_unique):
    unique_values = df[column_name].unique()
    sampled_values = np.random.choice(unique_values, size=num_unique, replace=False)
    filtered_df = df[df[column_name].isin(sampled_values)]
    return filtered_df

In [ ]:
def plot_data(filename,combined_df=df_all):
    filtered_df = combined_df[combined_df['FileName'] == filename]
    plt.figure(figsize=(8, 7))
    fig, axs = plt.subplots(2, 1, figsize=(10, 7), sharex=True) 
    updated_observables = ['Vein_species_Hot', 'Vein_species_Cold']

   
    for ax, observable in zip(axs, updated_observables):
        ax.plot(filtered_df['Time'], filtered_df[observable], label=observable)
        ax.set_title(observable)
        ax.set_ylabel('Value')

   
    axs[-1].set_xlabel('Time (minutes)')

    plt.tight_layout()
    plt.show()

plot_data('SimDataResults_497.csv')

In [ ]:
feature_columns = [col for col in df_all.columns if  col not in ['Unnamed: 0', 'FileName', 'Time']]
df_all['DateTime'] = pd.to_datetime('2020-01-01') + pd.to_timedelta(df_all['Time'], unit='m')
df_all.set_index('DateTime', inplace=True)
df_transf=pd.DataFrame()
unique_files = df_all['FileName'].unique()
for file in unique_files:
    sample_file_data = df_all[df_all['FileName'] == file][feature_columns]
    daily_aggregated_data = sample_file_data.resample('1440T').max()
    daily_aggregated_data['FileName']=file
    daily_aggregated_data.reset_index(drop=True, inplace=True)
    df_transf = pd.concat([df_transf, daily_aggregated_data])

df_transf.reset_index(drop=True, inplace=True)

# Start From Home if Restarted

In [5]:
#df_transf.to_csv(folder_path.replace('SimDataCSVs', 'df_trans_sim.csv'))
folder_path = '/Users/tbin/Library/CloudStorage/GoogleDrive-contact@bineshkumar.me/My Drive/Colab Notebooks/UBC_PSMA_Model_Simulator_Data/SimDataCSVs'
df_transf=pd.read_csv(folder_path.replace('SimDataCSVs','df_trans_sim.csv'))

In [6]:
df_transf['TIAOARTotal']=df_transf['observable_TIA_Kidney'] + df_transf['observable_TIA_SG'] + df_transf['observable_TIA_Liver'] + df_transf['observable_TIA_Spleen'] + df_transf['observable_TIA_RedMarrow']

In [7]:
numerical_features = ['parameter_Tumor1VolumeCoeff', 'parameter_bodySurfaceArea', 'parameter_bodyWeight', 'Vein_species_Cold', 'parameter_RepeatTimeInterval', 'parameter_lambdaPhys', 'Vein_species_Hot', 'parameter_numberPerNanomole', 'parameter_RepeatCount']
target_col='observable_TumorTotalHot'

In [8]:
df_transf[numerical_features + [target_col]].isna().sum()

parameter_Tumor1VolumeCoeff     1036
parameter_bodySurfaceArea       1036
parameter_bodyWeight            1036
Vein_species_Cold               1036
parameter_RepeatTimeInterval    1036
parameter_lambdaPhys            1036
Vein_species_Hot                1036
parameter_numberPerNanomole     1036
parameter_RepeatCount           1036
observable_TumorTotalHot        1036
dtype: int64

In [9]:
df_transf = df_transf[~df_transf[target_col].isnull()]

In [10]:
X=df_transf[numerical_features]
y=df_transf[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
models_to_run = [Ridge(), SVR(), RandomForestRegressor(), xgb.XGBRegressor(), lgb.LGBMRegressor()]

model_parm_search = [
    { # Ridge
     'alpha': Real(1e-5, 100, prior='log-uniform'),
     "solver": Categorical(['svd', 'cholesky', 'lsqr', 'sag']),
     'fit_intercept': Categorical([True, False]),
    },
    
    { # SVR
     'C': Real(1e-6, 1e+6, prior='log-uniform'),
     "kernel": Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
     'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
     "epsilon": Real(0, 1),
    },

{ # RandomForestRegressor
 'max_depth': Integer(3, 12),
 'n_estimators': Integer(100, 1000),
 'max_features': Categorical(['sqrt', 'log2', None]),  # Adjusted here
},

    { # XGBRegressor
     'learning_rate': Real(1e-4, 1.0, 'log-uniform'),
     'colsample_bytree': Real(0.1, 0.9),
     'n_estimators': Integer(100, 1000),
     'reg_alpha': Real(1, 1.5, 'log-uniform'),
     'reg_lambda': Real(1, 1.5, 'log-uniform'),
    },

    { # LGBMRegressor
     'learning_rate': Real(1e-4, 1.0, 'log-uniform'),
     'n_estimators': Integer(100, 1000),
     'max_depth': Integer(3, 12),
     'reg_alpha': Real(1, 1.5, 'log-uniform'),
     'reg_lambda': Real(1, 1.5, 'log-uniform'),
    }
]

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
cv = KFold(n_splits=5, shuffle=True, random_state=7)
np.int=int
for i, model in enumerate(models_to_run):
   # build the Bayesian search model
   bs = BayesSearchCV(
       estimator=model,
       search_spaces=model_parm_search[i],
       cv = cv,
       n_iter = 20,
       n_jobs = -1, 
       scoring = "r2",
       #return_train_score=True,
       random_state = 2
   )

   bs.fit(X_train, y_train)
   
   model    = bs.best_estimator_
   test_preds = model.predict(X_test)
   train_preds = model.predict(X_train)

    
   r2_kfolds = cross_val_score(model, X_train, y_train, cv = cv, n_jobs=-1, scoring='r2')
   rmse_kfolds = cross_val_score(model, X_train, y_train, cv = cv, n_jobs=-1, scoring='neg_root_mean_squared_error')

   print("Model: ", model)
   print('\nThe 5-CV rmse Score was:', rmse_kfolds.mean()) 
   print('With a standard deviation of:', rmse_kfolds.std())
   print('Test rmse socre: %.2f' %mse(y_test, test_preds, squared=False))

   print('\nThe 5-CV R2 Score was:', r2_kfolds.mean()) 
   print('With a standard deviation of:', r2_kfolds.std())
   print("Test R2 Score : %.2f" %r2_score(y_test, test_preds))
   print("Train R2 Score : %.2f" %r2_score(y_train, train_preds))

Model:  Ridge(alpha=100.0, solver='cholesky')

The 5-CV rmse Score was: -0.020945197976233666
With a standard deviation of: 0.0008436017266379932
Test rmse socre: 0.02

The 5-CV R2 Score was: 0.3361050257964767
With a standard deviation of: 0.015510468333860999
Test R2 Score : 0.35
Train R2 Score : 0.34
Model:  SVR(C=59.548319327653964, epsilon=0.08612990629669139,
    gamma=0.0010148863931733528, kernel='sigmoid')

The 5-CV rmse Score was: -0.08363660196833793
With a standard deviation of: 0.00029219041653640254
Test rmse socre: 0.08

The 5-CV R2 Score was: -9.648045210034782
With a standard deviation of: 1.0094616779071421
Test R2 Score : -10.41
Train R2 Score : -9.55
